# Airports Role in Flight Delays

---


In [ ]:
# General Imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# R1.1 Imports
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import roc_curve, precision_recall_curve, roc_auc_score

#R1.2 Imports
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

#R1.3 Imports

## Data Preperation

There is special categories in these models, so the data must be prepared and classified before creating a model.


In [ ]:
df = pd.read_csv("../data/ORD_11_21-11-22_model.csv")

In [ ]:
# Two class subset
# df_x = df[['DOT_ID_Marketing_Airline', 'OriginAirportSeqID', 'DepTimeBlk', 'DepDelay', 'DestAirportSeqID', 'ArrTimeBlk', 'ArrDelay', 'Distance',
#                   'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'TaxiIn', 'TaxiOut']]
# df_y = df['DepDel15']

In [ ]:
df = df['ArrDelay'] >= 15

In [ ]:
df['AirportFault'] = df['CarrierDelay'] > df['ArrDelay']

## Logisitic Regression

## Random Forrest Decision Tree